# 多帧图像转为视频

In [ ]:
import cv2
import os

# 图像文件夹路径
image_folder = "visualResult/CoDeFv1/beauty_1"
fps = 30.0

# 视频输出路径和名称
video_name = "visualResult/CoDeFv1/beauty_1.avi"

# 获取图像文件夹中的所有图像文件名
images = [img for img in os.listdir(image_folder)]

# 读取第一张图像，获取图像尺寸
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# 使用cv2.VideoWriter创建视频写入对象
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), fps, (width, height))

# 将每个图像逐帧写入视频
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

# 关闭视频写入对象
video.release()


# 采样图像尺寸为指定大小

In [ ]:
# 应用PIL
from PIL import Image

def resize_image(input_image_path, output_image_path, size):
    with Image.open(input_image_path) as image:
        resized_image = image.resize(size)
        resized_image.save(output_image_path)

# 示例用法
input_path = 'input.jpg'  # 输入图像路径
output_path = 'output.jpg'  # 输出图像路径
target_size = (800, 600)  # 目标大小，宽度为800像素，高度为600像素

resize_image(input_path, output_path, target_size)


In [ ]:
# 应用opencv
# 指定新的图像尺寸
import cv2

# 加载图像
image = cv2.imread('input_image.jpg')  # 替换为你的图像路径

new_width = 500
new_height = 300

# 采样图像
resized_image = cv2.resize(image, (new_width, new_height))
# 保存采样后的图像
cv2.imwrite('output_image.jpg', resized_image)  # 替换为你想保存的路径和文件名



# 光流warp

In [ ]:
def flow_warp(x, flow, interp_mode='bilinear', padding_mode='zeros', align_corners=True):
    """Warp an image or feature map with optical flow.

    Args:
        x (Tensor): Tensor with size (n, c, h, w).
        flow (Tensor): Tensor with size (n, h, w, 2), normal value.
        interp_mode (str): 'nearest' or 'bilinear'. Default: 'bilinear'.
        padding_mode (str): 'zeros' or 'border' or 'reflection'.
            Default: 'zeros'.
        align_corners (bool): Before pytorch 1.3, the default value is
            align_corners=True. After pytorch 1.3, the default value is
            align_corners=False. Here, we use the True as default.

    Returns:
        Tensor: Warped image or feature map.
    """
    assert x.size()[-2:] == flow.size()[1:3]
    _, _, h, w = x.size()
    # create mesh grid
    grid_y, grid_x = torch.meshgrid(torch.arange(0, h).type_as(x), torch.arange(0, w).type_as(x))
    grid = torch.stack((grid_x, grid_y), 2).float()  # W(x), H(y), 2
    grid.requires_grad = False

    vgrid = grid + flow
    # scale grid to [-1,1]
    vgrid_x = 2.0 * vgrid[:, :, :, 0] / max(w - 1, 1) - 1.0
    vgrid_y = 2.0 * vgrid[:, :, :, 1] / max(h - 1, 1) - 1.0
    vgrid_scaled = torch.stack((vgrid_x, vgrid_y), dim=3)
    output = F.grid_sample(x, vgrid_scaled, mode=interp_mode, padding_mode=padding_mode, align_corners=align_corners)

    # TODO, what if align_corners=False
    return output

# 裁剪任意尺寸的patch

In [ ]:
import torch

import numpy as np

def split(input_tensor, patch_size, overlap):
    patch_h = patch_size[0]
    patch_w = patch_size[1]
    b,c,h,w = input_tensor.shape
    h_space = np.arange(0, h-patch_h+1, patch_h-overlap)
    if h - (h_space[-1] + patch_h) > 0:
        h_space = np.append(h_space, h-patch_h)
    w_space = np.arange(0, w-patch_w+1, patch_w-overlap)
    if w - (w_space[-1] + patch_w) > 0:
        w_space = np.append(w_space, w-patch_w)
    
    patches = []
    for x in h_space:
        tmp = []
        for y in w_space:
            tensor = input_tensor[:,:,x:x+patch_h,y:y+patch_w]
            tmp.append(tensor)
            # print(x,y, tensor.shape)
        patches.append(tmp)
    return patches, h_space, w_space

def compose(input_tensor, patches, h_space, w_space, scale_factor ,is_latent=True):
    """
    # h_space: array([   0,  492,  984, 1476, 1968, 2460, 2952, 3444, 3584])
    # w_space: array([   0,  492,  984, 1476, 1968, 2460, 2560])
    """
    b,c,h,w = input_tensor.shape
    patch_h = patches[0][0].shape[-1]
    patch_w = patches[0][0].shape[-2]
    h_space = [int(h*scale_factor) for h in h_space]
    w_space = [int(w*scale_factor) for w in w_space]
    h = int(h * scale_factor)
    w = int(w * scale_factor)
    if is_latent:
        h_space = [h//8 for h in h_space]
        w_space = [w//8 for w in w_space]
        h = h // 8
        w = w // 8
        c = patches[0][0].shape[1]
    
    
    # 组合
    output_tensor = torch.zeros(b, c, h, w).cpu()
    for i,x in enumerate(h_space):
        for j,y in enumerate(w_space):
            output_tensor[:,:,x:x+patch_h, y:y+patch_w] += patches[i][j].cpu()
    # 对重叠区域进行平均处理
    pred_x_end = 0
    for x in h_space:
        if x == 0:
            pred_x_end = x + patch_h
            continue
        output_tensor[:,:,x:pred_x_end,:] /= 2
        pred_x_end = x + patch_h
    pred_y_end = 0
    for y in w_space:
        if y == 0:
            pred_y_end = y + patch_w
            continue
        output_tensor[:,:,:,y:pred_y_end] /= 2
        pred_y_end = y + patch_w
    return output_tensor

import cv2
import torch
def read_img(img_path):
    """
    return:
        b c h w
    """
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB).astype("float32")
    img = torch.from_numpy(img.transpose(2,0,1)).float()/255.0
    return img.unsqueeze(0)

import torchvision.transforms.functional as tf
def tensor2img(tensor,name):
    """
    tensor:
        c h w
    """
    image = tf.to_pil_image(tensor)
    image.save(name)

import os

def getSplitBox(img, patch_size, overlap):
    b,c,h,w = img.shape
    h_space = np.arange(0, h-patch_size+1, patch_size-overlap)
    if h - (h_space[-1] + patch_size) > 0:
        h_space = np.append(h_space, h-patch_size)
    w_space = np.arange(0, w-patch_size+1, patch_size-overlap)
    if w - (w_space[-1] + patch_size) > 0:
        w_space = np.append(w_space, w-patch_size)
    return h_space, w_space

def composeFromFiles(patch_path, patch_size, overlap):
    patch_list = os.listdir(patch_path)
    img = torch.zeros(1,3, 3072, 4096)
    h_space, w_space = getSplitBox(img, patch_size, overlap)
    n_h = len(h_space)
    n_w = len(w_space)
    patches = [[0]*n_w for _ in range(n_h)]
    print(len(patch_list))
    for p in patch_list:
        print(p)
        p_name,ext = p.split(".")
        if p_name == "compose":
            continue
        h,w = p_name.split("_")
        print(h, w)
        h = int(h)
        w = int(w)
        patches[h][w] = read_img(os.path.join(patch_path, p))
    compose_img = compose(img, patches, h_space, w_space, patch_size ,1, False)
    tensor2img(compose_img[0], os.path.join(patch_path, f"patch_compose.png"))

# input_tensor = torch.randn(1,3,4096,3075)
input_tensor = read_img("/data2/lvjiangtao/vivo/ModelZoo/SUPIR/ljtdir/input_image/SUPIR_maskGaussianGuidedFilter.png")
patch_size= 256
overlap = 20
scale_factor = 1024 // patch_size
patches,h_space,w_space = split(input_tensor, patch_size, overlap)
# h_space: array([   0,  492,  984, 1476, 1968, 2460, 2952, 3444, 3584])
# w_space: array([   0,  492,  984, 1476, 1968, 2460, 2560])
new_patches = []
for tmp in patches:
    new_tmp = []
    for patch in tmp:
        patch = torch.nn.functional.interpolate(patch, scale_factor=scale_factor)
        patch = torch.nn.functional.interpolate(patch, scale_factor=1/8)
        new_tmp.append(patch)
    new_patches.append(new_tmp)
print("patch shape:", new_patches[0][0].shape)
output_tensor = compose(input_tensor, new_patches, h_space.copy(), w_space.copy(), patch_size, scale_factor)
print("output tensor:", output_tensor.shape)
output_tensor = torch.nn.functional.interpolate(output_tensor, scale_factor=1/scale_factor)
output_tensor = torch.nn.functional.interpolate(output_tensor, scale_factor=8)
tensor2img(output_tensor[0], "/data2/lvjiangtao/vivo/ModelZoo/SUPIR/ljtdir/output_img/compose.png")
# print(output_tensor.shape)
# print(torch.allclose(input_tensor, output_tensor))